### Short-term memory

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver  


agent = create_agent(
    model="google_genai:gemini-2.5-flash-lite",
    # [get_user_info],
    # checkpointer=InMemorySaver(),  
)

agent.invoke(
    {"messages": [{"role": "user", "content": "Hi! My name is Bob."}]},
    {"configurable": {"thread_id": "1"}},  
)

{'messages': [HumanMessage(content='Hi! My name is Bob.', additional_kwargs={}, response_metadata={}, id='efdc3c67-5311-41d6-ab8b-a7c1150314d3'),
  AIMessage(content="Hi Bob! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--00552fea-f46f-4352-a0ab-5525aec9171e-0', usage_metadata={'input_tokens': 8, 'output_tokens': 18, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}})]}

In [ ]:
agent.invoke(
    {"messages": [{"role": "user", "content": "내 이름이 뭐였지?"}]},
    {"configurable": {"thread_id": "1"}},  
)

{'messages': [HumanMessage(content='내 이름이 뭐였지?', additional_kwargs={}, response_metadata={}, id='602eeb32-4b25-4022-bcc4-eb17c16ce8d5'),
  AIMessage(content='죄송합니다. 저는 대화 기록을 저장하지 않기 때문에 이전에 제가 당신의 이름을 알았는지, 혹은 당신의 이름이 무엇인지 알 수 없습니다.\n\n혹시 다시 알려주시겠어요?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--90757089-5c1f-4ae7-9fdb-c52c7f7dc211-0', usage_metadata={'input_tokens': 8, 'output_tokens': 44, 'total_tokens': 52, 'input_token_details': {'cache_read': 0}})]}

에이전트에 단기 메모리(스레드 수준 지속성)를 추가하려면 에이전트를 생성할 때 checkpointer 지정해야 합니다.

In [3]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver  


agent = create_agent(
    model="google_genai:gemini-2.5-flash-lite",
    # [get_user_info],
    checkpointer=InMemorySaver(),  
)

agent.invoke(
    {"messages": [{"role": "user", "content": "Hi! My name is Bob."}]},
    {"configurable": {"thread_id": "1"}},  
)

{'messages': [HumanMessage(content='Hi! My name is Bob.', additional_kwargs={}, response_metadata={}, id='b08bcb81-5bb0-4ac4-871f-62add6192b3b'),
  AIMessage(content="Hi Bob! It's nice to meet you. I'm a large language model, trained by Google. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--dd0bebcb-aa4a-4201-b530-3f74f718486b-0', usage_metadata={'input_tokens': 8, 'output_tokens': 30, 'total_tokens': 38, 'input_token_details': {'cache_read': 0}})]}

In [4]:
agent.invoke(
    {"messages": [{"role": "user", "content": "내 이름이 뭐였지?"}]},
    {"configurable": {"thread_id": "1"}},  
)

{'messages': [HumanMessage(content='Hi! My name is Bob.', additional_kwargs={}, response_metadata={}, id='b08bcb81-5bb0-4ac4-871f-62add6192b3b'),
  AIMessage(content="Hi Bob! It's nice to meet you. I'm a large language model, trained by Google. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--dd0bebcb-aa4a-4201-b530-3f74f718486b-0', usage_metadata={'input_tokens': 8, 'output_tokens': 30, 'total_tokens': 38, 'input_token_details': {'cache_read': 0}}),
  HumanMessage(content='내 이름이 뭐였지?', additional_kwargs={}, response_metadata={}, id='1206530c-93e0-4904-ae86-24247e84258f'),
  AIMessage(content='당신의 이름은 Bob이었습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_nam

기본적으로 에이전트는 AgentState 사용하여 단기 메모리, 특히 messages 키를 통한 대화 기록을 관리합니다.

AgentState 확장하여 필드를 추가할 수 있습니다. 

사용자 지정 상태 스키마는 state_schema 매개변수를 사용하여 create_agent 에 전달됩니다.

In [8]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver


class CustomAgentState(AgentState):  
    user_id: str
    preferences: dict

agent = create_agent(
    model="google_genai:gemini-2.5-flash-lite",
    # [get_user_info],
    state_schema=CustomAgentState,  
    checkpointer=InMemorySaver(),
)

# Custom state can be passed in invoke
result = agent.invoke(
    {
        "messages": [{"role": "user", "content": "Hello"}],
        "user_id": "user_123",  
        "preferences": {"theme": "dark"}  
    },
    {"configurable": {"thread_id": "1"}})

In [9]:
result

{'messages': [HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}, id='d1679f9e-4278-44ec-a4ac-6bf2d4a9b4b9'),
  AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--2c0626e9-0948-4319-b9f1-3a0d974445bd-0', usage_metadata={'input_tokens': 2, 'output_tokens': 9, 'total_tokens': 11, 'input_token_details': {'cache_read': 0}})],
 'user_id': 'user_123',
 'preferences': {'theme': 'dark'}}

#### Trim messages  메시지 트리밍

@before_model 미들웨어 데코레이터 사용한 메시지 기록 정리

In [ ]:
from langchain.messages import RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import before_model
from langgraph.runtime import Runtime
from langchain_core.runnables import RunnableConfig
from typing import Any


@before_model
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Keep only the last few messages to fit context window."""
    messages = state["messages"]

    if len(messages) <= 3:
        return None  # No changes needed

    first_msg = messages[0]
    # first_msg = ""
    recent_messages = messages[-3:] if len(messages) % 2 == 0 else messages[-4:]
    # recent_messages = messages[-1:] if len(messages) % 2 == 0 else messages[-2:]
    new_messages = [first_msg] + recent_messages

    return {
        "messages": [
            RemoveMessage(id=REMOVE_ALL_MESSAGES),
            *new_messages
        ]
    }

agent = create_agent(
    model="google_genai:gemini-2.5-flash-lite",
    # tools=tools,
    middleware=[trim_messages],
    checkpointer=InMemorySaver(),
)

config: RunnableConfig = {"configurable": {"thread_id": "1"}}

agent.invoke({"messages": "hi, my name is bob"}, config)
agent.invoke({"messages": "write a short poem about cats"}, config)
agent.invoke({"messages": "now do the same but for dogs"}, config)
final_response = agent.invoke({"messages": "what's my name?"}, config)

# final_response["messages"][-1].pretty_print()

"\n================================== Ai Message ==================================\n\nYour name is Bob. You told me that earlier.\nIf you'd like me to call you a nickname or use a different name, just say the word.\n"

In [17]:
final_response["messages"]

[HumanMessage(content='hi, my name is bob', additional_kwargs={}, response_metadata={}, id='edeb9ec8-8b61-4f31-ab73-3031b16b6c58'),
 AIMessage(content='A furry shadow, soft and sleek,\nWith eyes of emerald, secrets they speak.\nA gentle purr, a rumbling sound,\nOn silent paws, they softly bound.\n\nA playful leap, a twitching tail,\nThrough sunbeams they gracefully trail.\nThey nap and dream, then stretch and yawn,\nMasters of comfort, from dusk till dawn.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--faa8a12c-5590-4de7-81d8-f5036d06ee15-0', usage_metadata={'input_tokens': 34, 'output_tokens': 77, 'total_tokens': 111, 'input_token_details': {'cache_read': 0}}),
 HumanMessage(content='now do the same but for dogs', additional_kwargs={}, response_metadata={}, id='a1bd48fc-d

#### Delete messages  메시지 삭제

그래프 상태에서 메시지를 삭제하여 메시지 기록을 관리할 수 있습니다.

이 기능은 특정 메시지를 제거하거나 전체 메시지 기록을 지우려는 경우에 유용합니다.

그래프 상태에서 메시지를 삭제하려면 RemoveMessage 를 사용하면 됩니다.

In [24]:
from langchain.messages import RemoveMessage
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import after_model
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.runtime import Runtime
from langchain_core.runnables import RunnableConfig

agent = create_agent(
    model="google_genai:gemini-2.5-flash-lite",
    tools=[],
    system_prompt="Please be concise and to the point.",
    # middleware=[delete_old_messages], # * 사용하지 않은 경우
    checkpointer=InMemorySaver(),
)

config: RunnableConfig = {"configurable": {"thread_id": "1"}}

for event in agent.stream(
    {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
    config,
    stream_mode="values",
):
    print([(message.type, message.content) for message in event["messages"]])

for event in agent.stream(
    {"messages": [{"role": "user", "content": "what's my name?"}]},
    config,
    stream_mode="values",
):
    print([(message.type, message.content) for message in event["messages"]])

[('human', "hi! I'm bob")]
[('human', "hi! I'm bob"), ('ai', 'Hi Bob.')]
[('human', "hi! I'm bob"), ('ai', 'Hi Bob.'), ('human', "what's my name?")]
[('human', "hi! I'm bob"), ('ai', 'Hi Bob.'), ('human', "what's my name?"), ('ai', 'Your name is Bob.')]


In [25]:
agent.invoke(
    {"messages": [{"role": "user", "content": "what's my name?"}]},
    config,
)

{'messages': [HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}, id='998e138c-1596-4f38-8f5c-fcc16fe8faaf'),
  AIMessage(content='Hi Bob.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--f8297a9e-6ea4-470b-8065-b893ac1bbcac-0', usage_metadata={'input_tokens': 15, 'output_tokens': 3, 'total_tokens': 18, 'input_token_details': {'cache_read': 0}}),
  HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}, id='8b0a216e-fb30-4b6e-baae-91ba1bc09d85'),
  AIMessage(content='Your name is Bob.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'goo

In [37]:
@after_model
def delete_old_messages(state: AgentState, runtime: Runtime) -> dict | None:
    """Remove old messages to keep conversation manageable."""
    messages = state["messages"]
    if len(messages) > 2:
        # remove the earliest two messages
        return {"messages": [RemoveMessage(id=m.id) for m in messages[:2]]}
        # 모든 메시지 제거
        # return {"messages": [RemoveMessage(id=REMOVE_ALL_MESSAGES)]}
    
    return None

agent = create_agent(
    model="google_genai:gemini-2.5-flash-lite",
    tools=[],
    system_prompt="Please be concise and to the point.",
    middleware=[delete_old_messages], # * 사용한 경우
    checkpointer=InMemorySaver(),
)

config: RunnableConfig = {"configurable": {"thread_id": "1"}}

for event in agent.stream(
    {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
    config,
    stream_mode="values",
):
    print([(message.type, message.content) for message in event["messages"]])

# for event in agent.stream(
#     {"messages": [{"role": "user", "content": "what's my name?"}]},
#     config,
#     stream_mode="values",
# ):
#     print([(message.type, message.content) for message in event["messages"]])

[('human', "hi! I'm bob")]
[('human', "hi! I'm bob"), ('ai', 'Hello Bob.')]


In [38]:
agent.invoke(
    {"messages": [{"role": "user", "content": "what's my name?"}]},
    config,
)

{'messages': [HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}, id='35ae81d9-67b2-4df1-bf84-03ee3d6f0b3a'),
  AIMessage(content='Your name is Bob.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--e78bd44f-c1ce-4a1f-8f06-8c47e9ba91b4-0', usage_metadata={'input_tokens': 26, 'output_tokens': 5, 'total_tokens': 31, 'input_token_details': {'cache_read': 0}})]}

In [39]:
agent.invoke(
    {"messages": [{"role": "user", "content": "what's my name?"}]},
    config,
)

{'messages': [HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}, id='dd8264b0-272e-4a33-9b26-da734d73753a'),
  AIMessage(content='I do not have access to your personal information, including your name.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--ccff5630-bf10-40e6-bc32-8243f379916f-0', usage_metadata={'input_tokens': 28, 'output_tokens': 14, 'total_tokens': 42, 'input_token_details': {'cache_read': 0}})]}

#### Summarize messages  메시지 요약

채팅 모델을 사용하여 메시지 기록을 요약할 수 있습니다.

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig


checkpointer = InMemorySaver()

agent = create_agent(
    model="google_genai:gemini-2.5-flash",
    tools=[],
    middleware=[
        SummarizationMiddleware(
            model="google_genai:gemini-2.5-flash-lite",
            # max_tokens_before_summary=4000,  # Trigger summarization at 4000 tokens
            max_tokens_before_summary=300,  # Trigger summarization at 300 tokens
            messages_to_keep=20,  # Keep last 20 messages after summary
        )
    ],
    checkpointer=checkpointer,
)

config: RunnableConfig = {"configurable": {"thread_id": "1"}}
agent.invoke({"messages": "hi, my name is bob"}, config)
agent.invoke({"messages": "write a short poem about cats"}, config)
agent.invoke({"messages": "now do the same but for dogs"}, config)
final_response = agent.invoke({"messages": "what's my name?"}, config)

final_response["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob!
